# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 51 new papers today
          23 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/22 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.00806


extracting tarball to tmp_2212.00806...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.00806/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2212.01293... done.
Retrieving document from  https://arxiv.org/e-print/2212.01333


extracting tarball to tmp_2212.01333... done.
Retrieving document from  https://arxiv.org/e-print/2212.01352


extracting tarball to tmp_2212.01352...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.01369


extracting tarball to tmp_2212.01369... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00806-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00806) | **Distant Echoes of the Milky Way's Last Major Merger**  |
|| Vedant Chandra, et al. -- incl., <mark>Hans-Walter Rix</mark>, <mark>Dennis Zaritsky</mark> |
|*Appeared on*| *2022-12-05*|
|*Comments*| *19 pages, 10 figures. Submitted to ApJ*|
|**Abstract**| The majority of the Milky Way's stellar halo consists of debris from our Galaxy's last major merger, the Gaia-Sausage-Enceladus (GSE). In the past few years, stars from GSE have been kinematically and chemically studied in the inner $30$ kpc of our Galaxy. However, simulations predict that accreted debris could lie at greater distances, forming substructures in the outer halo. Here we derive metallicities and distances using Gaia DR3 XP spectra for an all-sky sample of luminous red giant stars, and map the outer halo with kinematics and metallicities out to $100$ kpc. We obtain follow-up spectra of stars in two strong overdensities - including the previously identified Outer Virgo Overdensity - and find them to be relatively metal-rich and on predominantly retrograde orbits, matching predictions from simulations of the GSE merger. We argue that these are apocentric shells of GSE debris, forming $60-90$ kpc counterparts to the $15-20$ kpc shells that are known to dominate the inner stellar halo. Extending our search across the sky with literature radial velocities, we find evidence for a coherent stream of retrograde stars encircling the Milky Way from $50-100$ kpc, in the same plane as the Sagittarius stream but moving in the opposite direction. These are the first discoveries of distant and structured imprints from the GSE merger, cementing the picture of an inclined and retrograde collision that built up our Galaxy's stellar halo. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00811-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00811) | **PHANGS-JWST First Results: A statistical view on bubble evolution in  NGC628**  |
|| Elizabeth J. Watkins, et al. -- incl., <mark>Kiana F. Henny</mark>, <mark>Kathryn Kreckel</mark>, <mark>Frank Bigiel</mark>, <mark>Jonathan Henshaw</mark>, <mark>Cinthya N. Herrera</mark>, <mark>Annie Hughes</mark>, <mark>Daizhong Liu</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-05*|
|*Comments*| *25 pages total, 13 Figures and 1 Table. Accepted for publication in ApJL as part of a PHANGS-JWST First Results Focus issue*|
|**Abstract**| The first JWST observations of nearby galaxies have unveiled a rich population of bubbles that trace the stellar feedback mechanisms responsible for their creation. Studying these bubbles therefore allows us to chart the interaction between stellar feedback and the interstellar medium, and the larger galactic flows needed to regulate star formation processes globally. We present the first catalog of bubbles in NGC628, visually identified using MIRI F770W PHANGS-JWST observations, and use them to statistically evaluate bubble characteristics. We classify 1694 structures as bubbles with radii between 6-552 pc. Of these, 31% contain at least one smaller bubble at their edge, indicating that previous generations of star formation have a local impact on where new stars form. On large scales, most bubbles lie near a spiral arm, and their radii increase downstream compared to upstream. Furthermore, bubbles are elongated in a similar direction to the spiral arm ridge-line. These azimuthal trends demonstrate that star formation is intimately connected to the spiral arm passage. Finally, the bubble size distribution follows a power-law of index $p=-2.2\pm0.1$, which is slightly shallower than the theoretical value by 1-3.5$\sigma$ that did not include bubble mergers. The fraction of bubbles identified within the shells of larger bubbles suggests that bubble merging is a common process. Our analysis therefore allows us to quantify the number of star-forming regions that are influenced by an earlier generation, and the role feedback processes have in setting the global star formation rate. With the full PHANGS-JWST sample, we can do this for more galaxies. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00812-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00812) | **PHANGS-JWST First Results: Multi-wavelength view of feedback-driven  bubbles (The Phantom Voids) across NGC 628**  |
|| Ashley T. Barnes, et al. -- incl., <mark>Kathryn Kreckel</mark>, <mark>Frank Bigiel</mark>, <mark>Janice C. Lee</mark>, <mark>Eva Schinnerer</mark>, <mark>Jonathan Henshaw</mark>, <mark>Jing Li</mark>, <mark>Daizhong Liu</mark>, <mark>Lukas Neumann</mark>, <mark>Francesca Pinna</mark> |
|*Appeared on*| *2022-12-05*|
|*Comments*| *12 pages total, 8 figures, and 1 table. Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| We present a high-resolution view of bubbles within The Phantom Galaxy (NGC 628); a nearby (~10Mpc), star-forming (~2Msun/yr), face-on (i~9deg) grand-design spiral galaxy. With new data obtained as part of the PHANGS-JWST treasury program, we perform a detailed case-study of two regions of interest, one of which contains the largest and most prominent bubble in the galaxy (The Phantom Void; over 1kpc in diameter), and the other being a smaller region that may be the precursor to such a large bubble (The Precursor Phantom Void). When comparing to matched resolution Halpha observations from the Hubble Space Telescope (HST), we see that the ionized gas is brightest in the shells of both bubbles, and is coincident with the youngest (~1Myr) and most massive (~100,000Msun) stellar associations. We also find an older generation (~20Myr) of stellar associations is present within the bubble of The Phantom Void. From our kinematic analysis of the HI, H2 (CO) and HII gas across The Phantom Void, we infer a high expansion speed of around 15 to 50km/s. The large size and high expansion speed of The Phantom Void suggest that the driving mechanism is sustained stellar feedback due to multiple mechanisms, where early feedback first cleared a bubble (as we observe now in The Precursor Phantom Void), and since then SNe have been exploding within the cavity, and have accelerated the shell. Finally, comparison to simulations shows a striking resemblance to our JWST observations, and suggests that such large-scale stellar feedback-driven bubbles should be common within other galaxies. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01281-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01281) | **A semiempirical approach to low-energy cosmic ray propagation in the  diffuse interstellar medium**  |
|| <mark>Riccardo Franceschi</mark>, Steven Neil Shore |
|*Appeared on*| *2022-12-05*|
|*Comments*| *8 pages, 13 figures*|
|**Abstract**| We investigate the ionization of the diffuse interstellar medium by cosmic rays by modeling their propagation along the wandering magnetic fields using a Monte Carlo method. We study how low-energy cosmic rays propagate in turbulent, translucent molecular clouds, and how they regulate the ionization and both lose and gain energy from the medium. As a test case, we use high spatial resolution (0.03 pc) CO maps of a well-studied high latitude translucent cloud, MBM 3, to model turbulence. The propagation problem is solved with a modified Monte Carlo procedure that includes trapping, energization, and ionization losses. In a homogeneous medium, trapping and re-energization do not produce a significant effect. In a nonuniform medium, particles can be trapped for a long time inside the cloud. This modifies the cosmic ray distribution due to stochastic acceleration at the highest energies (about 100 MeV). At lower energies, the re-energization is too weak to produce an appreciable effect. The change in the energy distribution does not significantly affect the ionization losses, so ionization changes are due to trapping effects. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01293-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01293) | **An investigation of the star-forming main sequence considering the  nebular continuum emission at low-z**  |
|| <mark>Henrique Miranda</mark>, et al. |
|*Appeared on*| *2022-12-05*|
|*Comments*| *13 pages, 10 figures and 3 tables. Forthcoming article in Astronomy and Astrophysics*|
|**Abstract**| The code FADO is the first publicly available population spectral synthesis tool that treats the contribution from ionised gas to the observed emission self-consistently. We study the impact of the nebular contribution on the determination of the star formation rate (SFR), stellar mass, and consequent effect on the star-forming main sequence (SFMS) at low redshift. We applied FADO to the spectral database of the SDSS to derive the physical properties of galaxies. As a comparison, we used the data in the MPA-JHU catalogue, which contains the properties of SDSS galaxies derived without the nebular contribution. We selected a sample of SF galaxies with H$\alpha$ and H$\beta$ flux measurements, and we corrected the fluxes for the nebular extinction through the Balmer decrement. We then calculated the H$\alpha$ luminosity to estimate the SFR. Then, by combining the stellar mass and SFR estimates from FADO and MPA-JHU, the SFMS was obtained. The H$\alpha$ flux estimates are similar between FADO and MPA-JHU. Because the H$\alpha$ flux was used as tracer of the SFR, FADO and MPA-JHU agree in their SFR. The stellar mass estimates are slightly higher for FADO than for MPA-JHU on average. However, considering the uncertainties, the differences are negligible. With similar SFR and stellar mass estimates, the derived SFMS is also similar between FADO and MPA-JHU. Our results show that for SDSS normal SF galaxies, the additional modelling of the nebular contribution does not affect the retrieved fluxes and consequentially also does not influence SFR estimators based on the extinction-corrected H$\alpha$ luminosity. For the stellar masses, the results point to the same conclusion. These results are a consequence of the fact that the vast majority of normal SF galaxies in the SDSS have a low nebular contribution. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01333-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01333) | **Diffuse $γ$-ray emission in Cygnus X: Comments to Yan & Pavaskar**  |
|| Ottavio Fornieri, <mark>Heshou Zhang</mark> |
|*Appeared on*| *2022-12-05*|
|*Comments*| *4 pages, 1 figure*|
|**Abstract**| It has been pointed out in arXiv:2211.17057 that our recent (published) paper might be revised, due to an incorrect evaluation of the diffusion coefficients, $D(E)$, employed in the calculations. Unfortunately, there is no indication as to where the incorrectness might be. Here we offer the opportunity to be more specific, by providing the community with the whole description of the equations involved in the calculation of $D(E)$, which is missing in the {\tt arXiv} note. In this context, we mention that no \textit{ad hoc} parameterisation has been used in our paper. Furthermore, any assumption on the injection mechanisms is explicitly described in the paper as an input factor and is obviously part of the modelisation procedure, hence the final outcome is subject to it. Finally, we discuss in a more broad context what, in this calculation of the diffusion coefficient, we believe is the key message. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01352-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01352) | **The SDSS-HET Survey of Kepler Eclipsing Binaries. A Sample of Four  Benchmark Binaries**  |
|| Kelly Hambleton, et al. -- incl., <mark>Suvrath Mahadevan</mark> |
|*Appeared on*| *2022-12-05*|
|*Comments*| **|
|**Abstract**| The purpose of this work is to extend a sample of accurately modeled, benchmark-grade eclipsing binaries with accurately determined masses and radii. We select four "well-behaved" Kepler binaries, KIC2306740, KIC4076952, KIC5193386 and KIC5288543, each with at least 8 double-lined spectra from the apogee instrument that is part of the Sloan Digital Sky Surveys III and IV, and from the Hobby-Eberly High Resolution Spectrograph. We obtain masses and radii with uncertainties of 2.5% or less for all four systems. Three of these systems have orbital periods longer than 9 days, and thus populate an under-sampled region of the parameter space for extremely well-characterized detached eclipsing binaries. We compare the derived masses and radii against MESA MIST isochrones to determine the ages of the systems. All systems were found to be coeval, showing that the results are consistent across MESA MIST and PHOEBE. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01369-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01369) | **Primordial Black Holes Place the Universe in Stasis**  |
|| Keith R. Dienes, et al. -- incl., <mark>Lucien Heurtier</mark> |
|*Appeared on*| *2022-12-05*|
|*Comments*| *10 pages, LaTeX, 2 figures*|
|**Abstract**| A variety of scenarios for early-universe cosmology give rise to a population of primordial black holes (PBHs) with a broad spectrum of masses. In this paper we demonstrate that the evaporation of PBHs in such scenarios has the potential to place the universe into an extended period of "stasis" during which the abundances of matter and radiation remain absolutely constant despite cosmological expansion. This surprising phenomenon can give rise to new possibilities for early-universe dynamics and lead to distinctive signatures of the evaporation of such PBHs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00815-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00815) | **The TeV Sun Rises: Discovery of Gamma rays from the Quiescent Sun with  HAWC**  |
|| <mark>R. Alfaro</mark>, et al. -- incl., <mark>R. Diaz Hernandez</mark>, <mark>S. Hernandez</mark>, <mark>J. Lee</mark>, <mark>J.T. Linnemann</mark> |
|*Appeared on*| *2022-12-05*|
|*Comments*| *15 pages, 8 figures*|
|**Abstract**| We report the first detection of a TeV gamma-ray flux from the solar disk (6.3$\sigma$), based on 6.1 years of data from the High Altitude Water Cherenkov (HAWC) observatory. The 0.5--2.6 TeV spectrum is well fit by a power law, dN/dE = $A (E/1 \text{ TeV})^{-\gamma}$, with $A = (1.6 \pm 0.3) \times 10^{-12}$ TeV$^{-1}$ cm$^{-2}$ s$^{-1}$ and $\gamma = -3.62 \pm 0.14$. The flux shows a strong indication of anticorrelation with solar activity. These results extend the bright, hard GeV emission from the disk observed with Fermi-LAT, seemingly due to hadronic Galactic cosmic rays showering on nuclei in the solar atmosphere. However, current theoretical models are unable to explain the details of how solar magnetic fields shape these interactions. HAWC's TeV detection thus deepens the mysteries of the solar-disk emission. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.00806.md
    + _build/html/tmp_2212.00806/./lzsel_map_0.png
    + _build/html/tmp_2212.00806/./lzsel_map_1.png
    + _build/html/tmp_2212.00806/./lzsel_map_2.png
    + _build/html/tmp_2212.00806/./lzsel_sim_map.png
    + _build/html/tmp_2212.00806/./mike_orbits.png
    + _build/html/tmp_2212.00806/./mike_iom.png
    + _build/html/tmp_2212.00806/./ovo_rrl_map.png
    + _build/html/tmp_2212.00806/./allsky.png
exported in  _build/html/2212.00811.md
    + _build/html/tmp_2212.00811/./ngc628_jwst_bubble_location-arms-inset3.png
    + _build/html/tmp_2212.00811/./ngc628_rgb-bold.png
    + _build/html/tmp_2212.00811/./angle_offset-draft2.png
exported in  _build/html/2212.00812.md
    + _build/html/tmp_2212.00812/./figures/fig03_v2.png
    + _build/html/tmp_2212.00812/./figures/fig07.png
    + _build/html/tmp_2212.00812/./figures/fig02_v2_compressed.png
exported in  _build/html/2212.01281.md
    + _build/html/tmp_2212.01281/./pictures/dishomo.jpg
    + _build/html

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\kms$}{km s^{-1}}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Distant Echoes of the Milky Way's Last Major Merger

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.00806-b31b1b.svg)](https://arxiv.org/abs/2212.00806)<mark>Appeared on: 2022-12-05</mark> - _19 pages, 10 figures. Submitted to ApJ_

</div>
<div id="authors">

Vedant Chandra, et al. -- incl., <mark>Hans-Walter Rix</mark>, <mark>Dennis~Zaritsky</mark>

</div>
<div id="abstract">

**Abstract:** \noindentThe majority of the Milky Way's stellar halo consists of debris from our Galaxy's last major merger, the\textit{Gaia}-Sausage-Enceladus (GSE).In the past few years, stars from GSE have been kinematically and chemically studied in the inner$30$kpc of our Galaxy.However, simulations predict that accreted debris could lie at greater distances, forming substructures in the outer halo.Here we derive metallicities and distances using\textit{Gaia}DR3 XP spectra for an all-sky sample of luminous red giant stars, and map the outer halo with kinematics and metallicities out to$100$kpc.We obtain follow-up spectra of stars in two strong overdensities --- including the previously identified Outer Virgo Overdensity --- and find them to be relatively metal-rich and on predominantly retrograde orbits, matching predictions from simulations of the GSE merger.We argue that these are apocentric shells of GSE debris, forming$60-90$kpc counterparts to the$15-20$kpc shells that are known to dominate the inner stellar halo.Extending our search across the sky with literature radial velocities, we find evidence for a coherent stream of retrograde stars encircling the Milky Way from$50-100$kpc, in the same plane as the Sagittarius Stream but moving in the opposite direction.These are the first discoveries of distant and structured imprints from the GSE merger, cementing the picture of an inclined and retrograde collision that built up our Galaxy's stellar halo.

</div>

<div id="div_fig1">

<img src="tmp_2212.00806/./lzsel_map_0.png" alt="Fig7.1" width="25%"/><img src="tmp_2212.00806/./lzsel_map_1.png" alt="Fig7.2" width="25%"/><img src="tmp_2212.00806/./lzsel_map_2.png" alt="Fig7.3" width="25%"/><img src="tmp_2212.00806/./lzsel_sim_map.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** On-sky distribution of distant 6D giant stars, split into three subsets of azimuthal angular momentum $L_{\mathrm{Z}}$ in units of $10^3 \text{kpc km s}^{-1}$: stars on prograde orbits, stars on isotropic orbits, and stars with retrograde orbits (see Figure \ref{fig:lz_selection}). The selection function of the underlying spectroscopic surveys, and our Galactic latitude cut to mask out the MW disk, is shaded in grey. Sagittarius stream stars selected via $L_\mathrm{Y}$ are shown in purple. Whereas the stars with prograde and isotropic orbits are uniformly distributed across the survey footprints --- apart from Sagittarius, which dominates these panels --- the retrograde stars show a suggestive track on the sky, to which we fit the great circle shown in red. This track coincides with the MIKE-N (Outer Virgo Overdensity) and MIKE-S (Pisces Overdensity) stars, whose median positions are overlaid. A red (purple) arrow indicates the on-sky orbital motion of the retrograde (Sagittarius) stars, emphasizing that the spatially overlapping debris moves in opposite directions. In the bottom right panel we show GSE-only particles from the merger simulation of \citetalias{Naidu2021}, selected to lie beyond 40 kpc. (*fig:lz_maps*)

</div>
<div id="div_fig2">

<img src="tmp_2212.00806/./mike_orbits.png" alt="Fig2.1" width="50%"/><img src="tmp_2212.00806/./mike_iom.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Top: Future (past) orbits for the MIKE-N (MIKE-S) stars targeted in the Outer Virgo Overdensity (Pisces Overdensity), integrated over 1 Gyr with arrows denoting the general direction of these orbits on-sky. Bottom: MIKE targets in $L_\mathrm{Z}$-$E_{\mathrm{tot}}$ space, with the \textit{Gaia} XP 6D giant sample shown for comparison (black dots). The density distribution for \citetalias{Naidu2021} GSE simulation particles selected from analogous sky regions are shown using contours with corresponding colors. The MIKE stars in both hemispheres are overwhelmingly retrograde. Note that that the MIKE-S stars had a proper motion selection criterion that boosts the likelihood of being retrograde, whereas the MIKE-N stars were blindly targeted within the Outer Virgo Overdensity ($\S$\ref{sec:analysis:ovo:specfollow}). (*fig:mike_iom*)

</div>
<div id="div_fig3">

<img src="tmp_2212.00806/./ovo_rrl_map.png" alt="Fig6.1" width="50%"/><img src="tmp_2212.00806/./allsky.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** The spatial distribution of outer halo substructure as seen by RRL and RGB stars. Top: All-sky distribution of \textit{Gaia} DR3 RRL beyond 40 kpc, with the overdensities studied in this work highlighted by red and blue boxes.
    Bottom: Density distribution of our all-sky \textit{Gaia} XP sample of RGB stars beyond 40 kpc. We overlay our spectroscopic targets in the MIKE-North (Outer Virgo Overdensity) and MIKE-South (Pisces Overdensity) regions. There are observed overdensities of RRL and RGB stars in both these locations, albeit weaker and more extended in the south. (*fig:allsky*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\HST}{{\it HST}}$
$\newcommand{\JWST}{{\it JWST}}$
$\newcommand{\micron}{\mum}$
$\newcommand{\}{pt}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\msun$}{M_\odot}$
$\newcommand{$\HST$}{{\it HST}}$
$\newcommand{$\JWST$}{{\it JWST}}$
$\newcommand{$\micron$}{\mum}$
$\newcommand{\}{pt}$</div>



<div id="title">

#  First Results: A statistical view on bubble evolution in NGC 628

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.00811-b31b1b.svg)](https://arxiv.org/abs/2212.00811)<mark>Appeared on: 2022-12-05</mark> - _25 pages total, 13 Figures and 1 Table. Accepted for publication in ApJL as part of a PHANGS-JWST First Results Focus issue_

</div>
<div id="authors">

Elizabeth J. Watkins, et al. -- incl., <mark><mark>Kiana F. Henny</mark></mark>, <mark><mark>Kathryn Kreckel</mark></mark>, <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Jonathan Henshaw</mark></mark>, <mark><mark>Cinthya N. Herrera</mark></mark>, <mark><mark>Annie Hughes</mark></mark>, <mark><mark>Daizhong Liu</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** The first$\JWST$observations of nearby galaxies have unveiled a rich population of bubbles that trace the stellar feedback mechanisms responsible for their creation. Studying these bubbles therefore allows us to chart the interaction between stellar feedback and the interstellar medium, and the larger galactic flows needed to regulate star formation processes globally. We present the first catalog of bubbles in NGC 628, visually identified using MIRI F770W PHANGS--$\JWST$observations, and use them to statistically evaluate bubble characteristics. We classify 1694 structures as bubbles with radii between 6--552 pc. Of these, 31\%contain at least one smaller bubble at their edge, indicating that previous generations of star formation have a local impact on where new stars form. On large scales, most bubbles lie near a spiral arm, and their radii increase downstream compared to upstream. Furthermore, bubbles are elongated in a similar direction to the spiral arm ridge-line.These azimuthal trends demonstrate that star formation is intimately connected to the spiral arm passage. Finally, the bubble size distribution follows a power-law of index$p=-2.2\pm0.1$, which is slightly shallower than the theoretical value by 1--3.5$\sigma$that did not include bubble mergers. The fraction of bubbles identified within the shells of larger bubbles suggests that bubble merging is a common process. Our analysis therefore allows us to quantify the number of star-forming regions that are influenced by an earlier generation, and the role feedback processes have in setting the global star formation rate. With the full PHANGS--$\JWST$sample, we can do this for more galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2212.00811/./ngc628_jwst_bubble_location-arms-inset3.png" alt="Fig8" width="100%"/>

**Figure 8. -** \textbf{Bubble locations identified in NGC 628 with multi-scale view zoom-ins providing a more detailed view}. \textbf{Left}: MIRI F770W band map of NGC 628 using a square root intensity stretch. Cyan ellipses show the location of the 1694 bubbles identified. Labeled lines indicate position of spiral arms derived by skeletonizing the arm environment masks from \cite{querejeta_stellar_2021}. Solid circle markers on arm spines indicate the start position of the arm. White boxes labeled \textit{a} and \textit{b} indicate locations of close-up panels on right side of this figure. \textbf{Right}: Same RGB as Figure \ref{fig:bub_rgb} but focused on the regions indicated on the left panel. Bars at bottom left indicates the physical size scale.  (*fig:bub_loc*)

</div>
<div id="div_fig2">

<img src="tmp_2212.00811/./ngc628_rgb-bold.png" alt="Fig6" width="100%"/>

**Figure 6. -** {\bf Multi-scale bubble population in NGC 628 revealed with $\JWST$ observations}. RGB illustration towards the center of NGC 628 highlighting bubble features and the bands used in their identification. Red traces MIRI F770W observations, green traces MUSE H$\alpha$ observations and blue traces \textit{B}-band $\HST$ observations. White scalebar indicates physical size equal to 1 kpc. (*fig:bub_rgb*)

</div>
<div id="div_fig3">

<img src="tmp_2212.00811/./angle_offset-draft2.png" alt="Fig2" width="100%"/>

**Figure 2. -** \textbf{Offset angles between elliptical bubbles and the spiral arm tangent angle.}\textbf{Top}: Solid black and blue lines with apertures show offset angles for bubbles with aspect ratios $>1$ and $>1.15$, respectively, for bubbles with an uncertainty in measured angle of $\pm\ang{5;;}$, resulting in a KDE-like distribution. The blue line has been re-normalized to match the area of the black line (i.e., from 301 bubbles to 654 bubbles) to make comparisons easier. Translucent regions indicate the uncertainty. Dashed red line shows the distribution of offset angles if bubbles have random PAs. \textbf{Bottom}: Same as blue line in top panel (i.e., AR$>$1.15) but for bubbles $>40$ pc upstream (black) and downstream (purple) from an arm. (*fig:ang_off*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\HII}{\ion{H}{II}}$
$\newcommand{\HI}{\ion{H}{I}}$
$\newcommand{\Ha}{H{\alpha}}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\JWST}{\rm{\it JWST}}$
$\newcommand{\HST}{\rm{\it HST}}$
$\newcommand{\Astrosat}{\rm{\it Astrosat}}$
$\newcommand{\ch}[1]{\textcolor{black}{ #1}}$
$\newcommand{\namea}{\rm The Phantom Void}$
$\newcommand{\nameb}{\rm The Precursor Phantom Void}$
$\newcommand{\}{kms}$
$\newcommand{\}{Kkms}$
$\newcommand{\}{msun}$
$\newcommand{\}{lsun}$
$\newcommand{\}{solyr}$
$\newcommand{\}{dyncm}$
$\newcommand{\}{ergs}$
$\newcommand{\}{cmsq}$
$\newcommand{\}{cmcb}$
$\newcommand{\}{gcmsq}$
$\newcommand{\}{gcmcb}$
$\newcommand{\}{Kcmcb}$
$\newcommand{\}{ntdp}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\HII$}{$\ion${H}{II}}$
$\newcommand{$\HI$}{$\ion${H}{I}}$
$\newcommand{$\Ha$}{H{\alpha}}$
$\newcommand{$\kms$}{\rm km s^{-1}}$
$\newcommand{$\JWST$}{\rm{\it JWST}}$
$\newcommand{$\HST$}{\rm{\it HST}}$
$\newcommand{$\Astrosat$}{\rm{\it Astrosat}}$
$\newcommand{$\ch$}[1]{\textcolor{black}{ #1}}$
$\newcommand{$\namea$}{\rm The Phantom Void}$
$\newcommand{$\nameb$}{\rm The Precursor Phantom Void}$
$\newcommand{\}{kms}$
$\newcommand{\}{Kkms}$
$\newcommand{\}{msun}$
$\newcommand{\}{lsun}$
$\newcommand{\}{solyr}$
$\newcommand{\}{dyncm}$
$\newcommand{\}{ergs}$
$\newcommand{\}{cmsq}$
$\newcommand{\}{cmcb}$
$\newcommand{\}{gcmsq}$
$\newcommand{\}{gcmcb}$
$\newcommand{\}{Kcmcb}$
$\newcommand{\}{ntdp}$</div>



<div id="title">

# PHANGS-JWST First Results: Multi-wavelength view of feedback-driven bubbles (The Phantom Voids) across NGC 628

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.00812-b31b1b.svg)](https://arxiv.org/abs/2212.00812)<mark>Appeared on: 2022-12-05</mark> - _12 pages total, 8 figures, and 1 table. Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ_

</div>
<div id="authors">

Ashley T. Barnes, et al. -- incl., <mark><mark>Kathryn Kreckel</mark></mark>, <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Janice C. Lee</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>, <mark><mark>Jonathan Henshaw</mark></mark>, <mark><mark>Jing Li</mark></mark>, <mark><mark>Daizhong Liu</mark></mark>, <mark><mark>Lukas Neumann</mark></mark>, <mark><mark>Francesca Pinna</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present a high-resolution view of bubbles within The Phantom Galaxy (NGC 628); a nearby ($\sim$10 Mpc), star-forming ($\sim$2\msunyr$^{-1}$), face-on ($i$$\sim$9$^{\circ}$) grand-design spiral galaxy.With new data obtained as part of the PHANGS-$\JWST$treasury program, we perform a detailed case-study of two regions of interest, one of which contains the largest and most prominent bubble in the galaxy  ($\namea$; over 1 kpc in diameter), and the other being a smaller region that may be the precursor to such a large bubble ($\nameb$).When comparing to matched resolution$\Ha$observations from the Hubble Space Telescope ($\HST$), we see that the ionized gas is brightest in the shells of both bubbles, and is coincident with the youngest ($\sim$1 Myr) and most massive ($\sim$10$^{5}$\msun) stellar associations.We also find an older generation ($\sim$20 Myr) of stellar associations is present within the bubble of$\namea$.From our kinematic analysis of the HI, H$_2$(CO) and HII gas across$\namea$, we infer a high expansion speed of around 15 to 50$\kms$.The large size and high expansion speed of$\namea$suggest that the driving mechanism is sustained stellar feedback due to multiple mechanisms, where early feedback first cleared a bubble (as we observe now in$\nameb$), and since then SNe have been exploding within the cavity, and have accelerated the shell.Finally, comparison to simulations shows a striking resemblance to our$\JWST$observations, and suggests that such large-scale stellar feedback-driven bubbles should be common within other galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2212.00812/./figures/fig03_v2.png" alt="Fig7" width="100%"/>

**Figure 7. -** {\bf Comparison of a feedback-driven bubble, and its host stellar population for the Phantom Void and the Precursor Phantom Void (also shown in Fig. \ref{fig2}).} From left to right in upper panels, we show the $\JWST$ 7.7 \micron, $\HST$$\Ha$ and the near-UV (F275W) filters. Overlaid as colored contours on the upper right panel is the age of the stellar associations (at a scale of 32 pc; see \citealp{Deger2020}). From left to right in center panels are the {\it Spitzer 8 \micron}\citep{Dale2009}, MUSE $\Ha$\citep{Emsellem2022}, and Astrosat NUV (Hassani et al. in prep) observations. In the bottom panels are the ALMA CO \citep{leroy_phangs-alma_2021}, VLA HI \citep{Walter2008}, and HST broadband \citep{Lee2021} observations.
    The white dashed line on all panels denotes the bubble and shell of each region (see Fig. \ref{fig5}). We find that young and high-mass stellar associations ($<$ 20 Myr; $>$10$^{5}$\msun) exist within the bubble (particularly towards the boundaries), highlighting these as good candidates for driving the bubble expansion. (*fig3*)

</div>
<div id="div_fig2">

<img src="tmp_2212.00812/./figures/fig07.png" alt="Fig4" width="100%"/>

**Figure 4. -** {\bf Distribution of intensity and stellar population in the Phantom Void (see Fig. \ref{fig3}).} The upper panel shows the intensity distribution of the $\JWST$ 7.7 \micron,  continuum subtracted $\Ha$ emission, $\HST$ NUV, ALMA CO, and VLA HI emission, as measured radially from the center of the bubble (see Fig. \ref{fig3}). We show the 99$^{\rm th}$ percentile of the intensity in each radial bin, normalized to the value of the maximum bin (this percentile is chosen instead e.g. the mean value to better highlight where the emission distributions peak). The bottom panel shows the radial age distribution of the stellar association catalog. The size of the points indicate the mass of the stellar associations.  In both panels we highlight the position of the $\nameb$. We find evidence that younger and more higher-mass clusters are preferentially located within the shell, with respect to inside the bubble. (*fig5*)

</div>
<div id="div_fig3">

<img src="tmp_2212.00812/./figures/fig02_v2_compressed.png" alt="Fig6" width="100%"/>

**Figure 6. -** {\bf The prominent bubble structures across The Phantom Galaxy (Messier 74 or NGC 628).} In all panels, we show an image produced from the 770W (blue), 1000W (green), and 1130W (red) band filters from the $\JWST$(Lee et al. in prep), and overlaid in orange is the continuum subtracted $\HST$-$\Ha$. The faded circles and ellipses show the positions of the bubbles from \cite{WATKINS_PHANGSJWST}. (*fig2*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.01281-b31b1b.svg)](https://arxiv.org/abs/2212.01281)<mark>Appeared on: 2022-12-05</mark> - _8 pages, 13 figures_

</div>
<div id="authors">

<mark>Riccardo Franceschi</mark>, and
        Steven N. Shore

</div>
<div id="abstract">

**Abstract:** We investigate the ionization of the diffuse interstellar medium by cosmic rays by modeling their propagation along the wandering magnetic fields using a Monte Carlo method. We explore how particle trapping and second-order Fermi processes affect the ionization of the medium.We study how low-energy cosmic rays propagate in turbulent, translucent molecular clouds, and how they regulate the ionization and both lose and gain energy from the medium.As a test case, we use high spatial resolution (0.03 pc) CO maps of a well-studied high latitude translucent cloud, MBM 3, to model turbulence.  The propagation problem is solved with a modified Monte Carlo procedure that includes trapping, energization, and ionization losses.In the homogeneous medium, trapping and re-energization do not produce a significant effect. In the nonuniform medium, particles can be trapped for a long time inside the cloud. This modifies the cosmic ray distribution due to stochastic acceleration at the highest energies ($\sim$100 MeV). At lower energies, the re-energization is too weak to produce an appreciable effect. The change in the energy distribution does not significantly affect the ionization losses, so ionization changes are due to trapping effects.Our Monte Carlo approach to cosmic ray propagation is an alternative method for solving the transport equation. This approach can be benchmarked to gas observations of molecular clouds. Using this approach, we demonstrate that stochastic Fermi acceleration and particle trapping occurs in inhomogeneous clouds, significantly enhancing their ionization.

</div>

<div id="div_fig1">

<img src="tmp_2212.01281/./pictures/dishomo.jpg" alt="Fig6" width="100%"/>

**Figure 6. -** Configuration of the medium used to investigate the effects of inhomogeneities, represented by the gray regions. (*fig:dishomo*)

</div>
<div id="div_fig2">

<img src="tmp_2212.01281/./pictures/trapping_100.png" alt="Fig7" width="100%"/>

**Figure 7. -** Histogram of 100 MeV protons propagating in an inhomogeneous medium. The top plot represents the particles on the injection face reflected by the medium. The middle plots are at increasing depth, and the bottom represents transmission. The third and fourth plots from the top represent the interfaces with the inhomogeneous region. For comparison, the gray area shows the particle energy range in a homogeneous medium. The blue and red points do and do not include the effects of re-energization, respectively. The flat distribution in the lower energy range is the signature of trapping-enhanced ionization losses and small re-energization. (*fig:100_trapping*)

</div>
<div id="div_fig3">

<img src="tmp_2212.01281/./pictures/homo_10.png" alt="Fig3" width="100%"/>

**Figure 3. -** Histogram of 10 MeV protons propagating in a uniform medium. The top plot represents the particles on the injection face reflected by the medium. The middle plots are at increasing depth, and the bottom represents transmission. The red lines show a power-law fit of the particle distributions. (*fig:10mev_homo*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Constraining the turbulence and the dust disk in IM Lup: onset of planetesimal formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.01291-b31b1b.svg)](https://arxiv.org/abs/2212.01291)<mark>Appeared on: 2022-12-05</mark> - _15 pages, 16 figures, accepted by Astronomy & Astrophysics_

</div>
<div id="authors">

<mark>Riccardo Franceschi</mark>, Tilman Birnstiel, <mark>Thomas Henning</mark>, <mark>Anirudh Sharma</mark>

</div>
<div id="abstract">

**Abstract:** Observations of protoplanetary disks provide information on planet formation and the reasons for the diversity of planetary systems. The key to understanding planet formation is the study of dust evolution from small grains to pebbles. Smaller grains ($\sim 1 \;\mu m$) are well-coupled to the gas dynamics, and their distribution is significantly extended above the disk midplane. Larger grains settle much faster and are efficiently formed only in the midplane. By combining near-infrared polarized light and millimeter observations, it is possible to constrain the spatial distribution of both the small and large grains.We aim to construct detailed models of the size distribution and vertical/radial structure of the dust particles in protoplanetary disks based on observational data. In particular, we are interested in recovering the dust distribution in the IM Lup protoplanetary disk.We create a physical model for the dust distribution of protoplanetary disks. We then simulate the radiative transfer of the millimeter continuum and the near-infrared polarized radiation. Using a Markov chain Monte Carlo method, we compare the derived images to the observations available for the IM Lup disk to constrain the best physical model for IM Lup and to recover the vertical grain size distribution.The millimeter and near-infrared emission tightly constrain the dust mass and grain size distribution of our model. We find size segregation in the dust distribution, with millimeter-sized grains in the disk midplane. These grains are efficiently formed in the disk, possibly by sedimentation-driven coagulation, in accord with the short settling timescales predicted by our model. This also suggests a high dust-to-gas ratio at smaller radii in the midplane, possibly triggering streaming instabilities and planetesimal formation in the inner disk. We obtain a turbulent$\alpha$parameter of$3 \times 10^{-3}.$

</div>

<div id="div_fig1">

<img src="tmp_2212.01291/./optical_constants.png" alt="Fig1" width="100%"/>

**Figure 1. -** Medium optical constant for the dust composition in     Tab.\ref{table:dust_composition}\citep{Birnstiel18} (*fig:optical_constants*)

</div>
<div id="div_fig2">

<img src="tmp_2212.01291/./model_opac_1.png" alt="Fig2" width="100%"/>

**Figure 2. -** The model without a truncation in the dust distribution best reproducing the continuum data. This model does not include enough small grains to reproduce the scattered light observations. (*fig:model opac 1*)

</div>
<div id="div_fig3">

<img src="tmp_2212.01291/./model_opac_2.png" alt="Fig3" width="100%"/>

**Figure 3. -** The model without a truncation in the dust distribution best reproducing the scattered light data. This model does not have enough millimeter grains to match the observed continuum emission. (*fig:model opac 2*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

121  publications in the last 7 days.
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers